In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, GRU
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, '../utils')
import KMeansDateRange
import get_open_close
import timeseries

"""K is the number of different
industries/sectors in the stock market"""
k = 11

In [3]:
df = get_open_close.populate_values_efficient("2012-05-18", "2020-04-01")

In [4]:
clusters = KMeansDateRange.kmeans_automated("2012-05-18", "2020-04-01", k=k)

In [5]:
%%time
mean_absolute_loss = 0
"""Randomly sampling 20 stocks from each cluster"""
for cluster in range(0, k):
    sampled_stocks = clusters.loc[clusters['clusters'] == cluster].sample(n=20, random_state=1337)
    cluster_mae = 0
    
    model = Sequential()
    model.add(GRU(128, activation='tanh', input_shape=(14, 2), return_sequences=True))
    model.add(GRU(128, activation='tanh', return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1, activation="linear"))
    model.compile(optimizer='adam', loss='mae')
    
    stock_x_test = {}
    
    """Fit the data of each stock within a cluster on the RNN"""
    for index, stock in enumerate(sampled_stocks['symbols']):
        stock_data = df[stock]
        
        x_train, x_test = train_test_split(stock_data, test_size=0.3, shuffle=False)
        
        scaler = StandardScaler()
        x_train= scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)
        stock_x_test[stock] = x_test

        x_train, y_train = timeseries.get_timeseries_X_y(x_train)
        
        model.fit(x_train, y_train, validation_split=0.3, verbose=1,
                            initial_epoch=index*20, epochs=(index + 1)*20)
        
    for stock in stock_x_test:
        x_test, y_test = timeseries.get_timeseries_X_y(stock_x_test[stock])
        loss = model.evaluate(x_test, y_test)
        cluster_mae = cluster_mae + loss
    cluster_mae = cluster_mae/(sampled_stocks.shape[0])
    mean_absolute_loss = mean_absolute_loss + cluster_mae

print("Loss of LSTM: ", mean_absolute_loss/k)

2022-12-12 23:08:19.857852: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/20
30/30 [==============================] - 4s 47ms/step - loss: 0.1781 - val_loss: 0.0806
Epoch 2/20
30/30 [==============================] - 1s 23ms/step - loss: 0.0808 - val_loss: 0.0714
Epoch 3/20
30/30 [==============================] - 1s 21ms/step - loss: 0.0814 - val_loss: 0.0425
Epoch 4/20
30/30 [==============================] - 1s 21ms/step - loss: 0.0775 - val_loss: 0.0759
Epoch 5/20
30/30 [==============================] - 1s 22ms/step - loss: 0.0759 - val_loss: 0.0190
Epoch 6/20
30/30 [==============================] - 1s 22ms/step - loss: 0.0764 - val_loss: 0.0188
Epoch 7/20
30/30 [==============================] - 1s 21ms/step - loss: 0.0755 - val_loss: 0.0540
Epoch 8/20
30/30 [==============================] - 1s 22ms/step - loss: 0.0713 - val_loss: 0.0220
Epoch 9/20
30/30 [==============================] - 1s 22ms/step - loss: 0.0737 - val_loss: 0.0384
Epoch 10/20
30/30 [==============================] - 1s 22ms/step - loss: 0.0794 - val_loss: 0.0593
Epoch 11/

Epoch 83/100
30/30 [==============================] - 1s 22ms/step - loss: 0.0506 - val_loss: 0.0379
Epoch 84/100
30/30 [==============================] - 1s 21ms/step - loss: 0.0490 - val_loss: 0.0506
Epoch 85/100
30/30 [==============================] - 1s 21ms/step - loss: 0.0488 - val_loss: 0.0501
Epoch 86/100
30/30 [==============================] - 1s 22ms/step - loss: 0.0491 - val_loss: 0.0383
Epoch 87/100
30/30 [==============================] - 1s 24ms/step - loss: 0.0507 - val_loss: 0.0329
Epoch 88/100
30/30 [==============================] - 1s 21ms/step - loss: 0.0464 - val_loss: 0.0517
Epoch 89/100
30/30 [==============================] - 1s 22ms/step - loss: 0.0511 - val_loss: 0.0384
Epoch 90/100
30/30 [==============================] - 1s 23ms/step - loss: 0.0491 - val_loss: 0.0379
Epoch 91/100
30/30 [==============================] - 1s 23ms/step - loss: 0.0493 - val_loss: 0.0457
Epoch 92/100
30/30 [==============================] - 1s 22ms/step - loss: 0.0458 - val_los

30/30 [==============================] - 1s 21ms/step - loss: 0.1263 - val_loss: 0.1367
Epoch 164/180
30/30 [==============================] - 1s 21ms/step - loss: 0.1327 - val_loss: 0.1387
Epoch 165/180
30/30 [==============================] - 1s 22ms/step - loss: 0.1294 - val_loss: 0.1291
Epoch 166/180
30/30 [==============================] - 1s 23ms/step - loss: 0.1227 - val_loss: 0.1305
Epoch 167/180
30/30 [==============================] - 1s 22ms/step - loss: 0.1254 - val_loss: 0.1368
Epoch 168/180
30/30 [==============================] - 1s 22ms/step - loss: 0.1269 - val_loss: 0.1306
Epoch 169/180
30/30 [==============================] - 1s 22ms/step - loss: 0.1274 - val_loss: 0.1306
Epoch 170/180
30/30 [==============================] - 1s 22ms/step - loss: 0.1250 - val_loss: 0.1276
Epoch 171/180
30/30 [==============================] - 1s 22ms/step - loss: 0.1218 - val_loss: 0.1300
Epoch 172/180
30/30 [==============================] - 1s 23ms/step - loss: 0.1236 - val_loss: 0

30/30 [==============================] - 1s 20ms/step - loss: 0.0320 - val_loss: 0.0854
Epoch 244/260
30/30 [==============================] - 1s 21ms/step - loss: 0.0316 - val_loss: 0.0824
Epoch 245/260
30/30 [==============================] - 1s 20ms/step - loss: 0.0306 - val_loss: 0.0893
Epoch 246/260
30/30 [==============================] - 1s 20ms/step - loss: 0.0332 - val_loss: 0.1077
Epoch 247/260
30/30 [==============================] - 1s 20ms/step - loss: 0.0341 - val_loss: 0.0897
Epoch 248/260
30/30 [==============================] - 1s 20ms/step - loss: 0.0315 - val_loss: 0.1262
Epoch 249/260
30/30 [==============================] - 1s 20ms/step - loss: 0.0312 - val_loss: 0.1198
Epoch 250/260
30/30 [==============================] - 1s 20ms/step - loss: 0.0324 - val_loss: 0.1192
Epoch 251/260
30/30 [==============================] - 1s 20ms/step - loss: 0.0317 - val_loss: 0.1040
Epoch 252/260
30/30 [==============================] - 1s 20ms/step - loss: 0.0315 - val_loss: 0

30/30 [==============================] - 1s 20ms/step - loss: 0.0702 - val_loss: 0.0148
Epoch 324/340
30/30 [==============================] - 1s 20ms/step - loss: 0.0685 - val_loss: 0.0197
Epoch 325/340
30/30 [==============================] - 1s 20ms/step - loss: 0.0707 - val_loss: 0.0214
Epoch 326/340
30/30 [==============================] - 1s 20ms/step - loss: 0.0646 - val_loss: 0.0069
Epoch 327/340
30/30 [==============================] - 1s 20ms/step - loss: 0.0691 - val_loss: 0.0327
Epoch 328/340
30/30 [==============================] - 1s 20ms/step - loss: 0.0701 - val_loss: 0.0096
Epoch 329/340
30/30 [==============================] - 1s 20ms/step - loss: 0.0683 - val_loss: 0.0248
Epoch 330/340
30/30 [==============================] - 1s 20ms/step - loss: 0.0658 - val_loss: 0.0257
Epoch 331/340
30/30 [==============================] - 1s 20ms/step - loss: 0.0656 - val_loss: 0.0152
Epoch 332/340
30/30 [==============================] - 1s 20ms/step - loss: 0.0658 - val_loss: 0

19/19 [==============================] - 0s 6ms/step - loss: 0.0509
Loss of LSTM:  0.08998981700278819
CPU times: user 13min 56s, sys: 4min 28s, total: 18min 24s
Wall time: 4min 30s
